In [306]:
import pandas as pd
import numpy as np
import collections
import copy
from pathlib import Path
import pdb
import datetime
from IPython.display import clear_output

In [2]:
PATH = Path("../data/shop")

In [3]:
class Cata:
    def __init__(self):
        self._cata = pd.DataFrame(columns=["name","price","url","purchases"])
        self._builder = ItemBuilder()
        
    def load_cata(self,name="current"):
        name = f"cata-{name}.pkl"
        fn = PATH / name
        self._cata = pd.read_pickle(fn)
        print(f"Loaded {name} successfully.")
        
    def save_cata(self,name,curr=True):
        name = f"cata-{name}.pkl"
        fn = PATH / name
        self._cata.to_pickle(fn)
        print(f"Saved {name} successfully")
        if curr:
            name = "cata-current.pkl"
            fnc = PATH / name
            self._cata.to_pickle(fnc)
            print("Saved {name} successfully")
            
    def new_item(self,attrs=[None]*3):
        self._builder.blank_item()
        try: assert isinstance(attrs,list)
        except: return "Attrs arg must be list(name,price,url)"

        if len(attrs) < 3:
            attrs = attrs + [None]*(3-len(attrs))
        self._builder.name(attrs[0])
        self._builder.price(attrs[1])
        self._builder.url(attrs[2])
        
    def register_item(self):
        self._cata = self._cata.append(self._builder._item.__dict__,ignore_index=True)
        idx = int(self._cata[self._cata["name"] == self._builder._item.name].index[0])
        print(f"Registered {self._builder._item.name} as id: {idx}")
        
    def get_id(self,name=None):
        if name:
            return int(self._cata[self._cata["name"] == name].index[0])
        else:
            return int(self._cata[self._cata["name"] == self._builder._item.name].index[0])
    
    def get_item(self,item_id=None):
        if item_id is not None:
            item_vals = self._cata.iloc[item_id].tolist()
            self.new_item(item_vals)
            return item_vals
        else:
            item_vals = self._cata.iloc[self.get_id()].tolist()
            self.new_item(item_vals)
            return item_vals
        
    def display_item(self, item_id):
        cata_vals = self._cata.iloc[item_id]
        if isinstance(cata_vals,pd.Series):
            return cata_vals.to_frame().T
        else: return cata_vals

class Builder():
    def __init__(self):
        self._item = None
    def blank_item(self):
        self._item = item()
    def del_item(self):
        self._item = None

class ItemBuilder(Builder):
    def name(self,name):
        self._item.name = name
    def price(self,price):
        self._item.price = price
    def url(self,url):
        self._item.url = url
        
class item():
    def __init__(self):
        self.url = None
        self.name = None
        self.price = None
        self.purchases = 0
        
    def __str__(self):
        return f"{self.name} | {self.price} | {self.url}"
    def L(self):
        return [self.name, self.price, self.url]

In [126]:
class Wishlist:
    def __init__(self,cata):
        self._wl = pd.DataFrame(columns=["c_id","priority","urgency","bucket","exclude"])
        self._cata = cata
        self._builder = WishBuilder()
        self._wl.c_id = self._wl.c_id.astype(int)
        
    def load_wl(self,name="current"):
        name = f"wl-{name}.pkl"
        fn = PATH / name
        self._wl = pd.read_pickle(fn)
        print(f"Loaded {name} successfully.")
        
    def save_wl(self,name,curr=True):
        name = f"wl-{name}.pkl"
        fn = PATH / name
        self._wl.to_pickle(fn)
        print(f"Saved {name} successfully")
        if curr:
            name = "wl-current.pkl"
            fnc = PATH / name
            self._wl.to_pickle(fnc)
            print(f"Saved {name} successfully")
    
    def make_wish(self, attrs):
        self._builder.new_wish()
        try: assert isinstance(attrs,list)
        except: return "Attrs arg must be list(id, priority, urgency)"
        self._builder.id(int(attrs[0]))
        self._builder.priority(attrs[1])
        self._builder.urgency(attrs[2])
        self.register_wish()
    
    def register_wish(self):
        self._wl = self._wl.append([self._builder._wish.__dict__], ignore_index=True)
        
    def display_wish(self,item_id):
        cata_vals = self._cata.display_item(item_id)
        wish_vals = self._wl.loc[self._wl["c_id"] == item_id]
        wish_vals = wish_vals.astype({"c_id":int})
#         print(wish_vals)
        return wish_vals.merge(cata_vals,left_on="c_id",right_index=True,sort=True)
    
    def display_wishes(self):
        cata_vals = cart._cata._cata.iloc[cart._wl._wl.c_id.tolist()]
        return cart._wl._wl.astype({"c_id":int}).merge(cata_vals,left_on="c_id",right_index=True,sort=True)
        
    def get_wish(self,wish_id=None):
        if wish_id is not None:
            wish_vals = self._wl.iloc[wish_id].tolist()
            self.make_wish(wish_vals)
            return wish_vals
        else:
            wish_vals = self._wl.iloc[self._builder._wish.id].tolist()
            self.make_wish(wish_vals)
            return wish_vals
        
    def cata(self):
        return self._cata._cata
    
    
class wBuilder():
    def __init__(self):
        self._wish = None
    def new_wish(self):
        self._wish = wish()
        
class WishBuilder(wBuilder):
    def id(self,id):
        self._wish.c_id = id 
    def priority(self, prio):
        self._wish.priority = prio
    def urgency(self, urgency):
        self._wish.urgency = urgency
    def bucket(self, bucket):
        self._wish.bucket = bucket
    def score(self):
        self._wish.score = self._wish.priority * self._s_w[0] + self._wish.urgency * self._s_w[1]
        
class wish():
    def __init__(self):
        self.c_id = None
        self.priority = None
        self.urgency = None
        self.bucket = None
        self.exclude = False
    def __str__(self):
        return f"{self.c_id} | {self.priority} | {self.urgency} | {self.bucket}"

In [323]:
class Shop(CartBuilder):
    def __init__(self,wl):
        self._wl = wl
        self._cata = wl._cata
        self.budget = b = [25,50,75]
        self._budget = [0] + b + [float("inf")]
        self.savings = self._savings = [5,10,15]
        self._cart = Cart()
        self._s_w = [.6,.8]    #score weights
        self._cart._c = self.new_cart()
    
    def cata(self):
        return self._wl._cata._cata
    
    def wl(self):
        return self._wl._wl
    
    def budget(self):
        return self._budget
    
    def display_wishes(self):
        return self._wl.display_wishes()
    
    def display_cart(self):
        return self._cart._c
    
    def print_savings(self):
        names = ["low","mid","high","wtf"]
        layout = "{!s:<7}" * len(self._savings)
        print("Remaining savings:")
        print(layout.format(*names[:len(self._savings)]))
        print(layout.format(*list(np.round(self._savings))))
    
    def _score(self, row):
        return row[1] * self._s_w[0] + row [2] * self._s_w[1]
    
    def score_all(self,df):
        df["score"] = df.apply(self._score,axis=1)
        return df
    
    def sort(self):
        df = self._wl.display_wishes()
        ls = list(range(1,len(self._budget)))
        self._wl._wl.bucket = pd.cut(df.price, self._budget, labels=ls, 
                                     include_lowest=True)
    def fill_bucket(self,bucket):
        self.sort()
        df = self.display_wishes()
        df = self.score_all(df)
        df.sort_values(by="score",inplace=True)
        buck = df.loc[df.bucket == bucket + 1]
        bud = rem = self.budget[bucket]
        for row in buck.itertuples():
#             print(row)
            if row.exclude is not True:
                cost = rem - row.price
                if cost >= 0 and int(row.c_id) not in self._cart._c.values:
                    self.add_cart(int(row.c_id))
                    rem -= cost
#                     print(rem)
                else: continue
        if rem == bud: print(f"No eligible items found for bucket {bucket +1}")
        else: self._savings[bucket] = rem
            
    def autoshop(self):
        df = self.display_wishes()
        for i in range(len(self.budget)):
            self.fill_bucket(i)
        print(self.display_cart())
        self.print_savings()
        self.reshuffle()
        
    def reshuffle(self):
        r_id = input("Reshuffle? ")
        r_id = r_id.strip()
        if r_id != "":
            clear_output()
            r_id = int(r_id)
            self.rm_cart(r_id,True)
            item = self._wl.display_wish(r_id)
            self._savings[item.bucket.values[0]-1] += item.price.values[0]
            self.fill_bucket(item.bucket.values[0]-1)
            print(self.display_cart())
            self.print_savings()
            self.reshuffle()
    
class CartBuilder():
    def new_cart(self):
        names = self._wl._wl.columns.tolist()
        names = names + self._cata._cata.columns.tolist()
        cart = pd.DataFrame(columns=names)
        return cart
    
    def clear_cart(self):
        self._cart._c = self.new_cart()
        
    def add_cart(self,c_id):
        self.sort()
        df = self._cart._c
        item = self._wl.display_wish(c_id)
        item["score"] = self._score(item.values.tolist()[0])
        self._cart._c = pd.concat([df,item], sort=False)
        print(f"Added {item.name.values[0]} to cart.")
        
    def rm_cart(self,c_id, exclu=False):
        item = self._cart._c.loc[self._cart._c["c_id"] == c_id]
        self._cart._c = self._cart._c.drop(item.index.tolist())
        if exclu:
            self._wl._wl.loc[self._wl._wl.c_id == c_id,"exclude"] = True
        print(f"Removed {item.name.values[0]} from cart.")
        
class Cart():
    def __init__(self):
        self._c = None
        self._l = None
    
    def log_cart(self, desc):
        log = [datetime.datetime, desc]
        self._l.append(log,axis=1)
    

In [324]:
c = Cata()
wl = Wishlist(c)
wl._cata.load_cata()
# wl.load_wl()
cart = Shop(wl)
# cart._budget.append(75)

alist = [(0,2,2),
         (2,1,2),
         (16,2,2),
         (28,0,1),
         (31,0,0),
         (11,1,1)]

for i,p,u in alist:
    wl.make_wish([i,p,u])
#     cart.add_cart(i)
cart.wl()
cart.sort()
cart.score_all(cart.wl())

Loaded cata-current.pkl successfully.


,c_id,priority,urgency,bucket,exclude,score
0,0,2,2,1,False,2.8
1,2,1,2,1,False,2.2
2,16,2,2,1,False,2.8
3,28,0,1,3,False,0.8
4,31,0,0,4,False,0.0
5,11,1,1,1,False,1.4


In [325]:
# cart.fill_bucket(0)
# cart.rm_cart(11,True)
# cart._savings
# cart.rm_cart(11,True)
cart.autoshop()
# wl.display_wishes()
# type(cart._savings[0])

Removed Scott funko from cart.
Added Shampoo to cart.
No eligible items found for bucket 1
  c_id priority urgency bucket exclude        name price  \
1    2        1       2      1   False   Car mount  9.99   
3   28        0       1      3   False  Upright Go    70   
2   16        2       2      1   False     Shampoo    25   

                                                 url  score  
1  https://www.amazon.com/Mpow-Washable-Strong-On...    2.2  
3  https://www.amazon.com/dp/B0747YHYZF/?coliid=I...    0.8  
2  https://www.amazon.com/dp/B002KG4EJK/?coliid=I...    2.8  
Remaining savings:
low    mid    high   
30.0   10.0   70.0   
Reshuffle? 


In [ ]:
## class Director():
#     def __init__(self, builder):
#         self._builder = builder
        
#     def construct_car(self):
#         self._builder.create_new_car()
#         self._builder.add_model()
#         self._builder.add_tires()
#         self._builder.add_engine()
        
#     def get_car(self):
#         return self._builder.car
    
# class eBuilder():
#     def __init__(self):
#         self.car = None
#     def create_new_car(self):
#         self.car = Car()
        
# class SkylarkBuilder(eBuilder):
#     def add_model(self):
#         self.car.model = "Skylark"
#     def add_tires(self):
#         self.car.tires = "Regular tires"
#     def add_engine(self):
#         self.car.engine = "Turbo Engine"
        
# class Car():
#     def __init__(self):
#         self.model = None
#         self.tires = None
#         self.engine = None
#     def __str__(self):
#         return f"{self.model} | {self.tires} | {self.engine}"

In [ ]:
build = SkylarkBuilder()
director = Director(build)
director.construct_car()
print(director.get_car())

In [ ]:
pd.DataFrame.append()